In [0]:
%run "/Repos/pakkiraiah.kandi@outlook.com/databricks-practice/Formula1/Includes/configurations"


In [0]:
%run "/Repos/pakkiraiah.kandi@outlook.com/databricks-practice/Formula1/Includes/common_functions"

In [0]:
#Read data for races from processed
races_df=spark.read.parquet(f"{processed_folder_path}/race")
races_df=races_df.withColumnRenamed("name","race_name").withColumnRenamed("race_timestamp","race_date")

#Read Results data from processed
results_df=spark.read.parquet(f"{processed_folder_path}/results").withColumnRenamed("time","race_time")\
    .withColumnRenamed("race_id","result_race_id")\
    .withColumnRenamed("driver_id","result_driver_id")\
    .withColumnRenamed("constructor_id","result_constructor_id")

#Read data for circuits from processed
circuits_df=spark.read.parquet(f"{processed_folder_path}/circuits")
circuits_df=circuits_df.withColumnRenamed("name","circuit_name").withColumnRenamed("location","circuit_location")

#Read data for constructors from processed
constructors_df=spark.read.parquet(f"{processed_folder_path}/constructors").withColumnRenamed("name","constructor_name")


#Read Drivers data from processed
drivers_df=spark.read.parquet(f"{processed_folder_path}/drivers").withColumnRenamed("name","driver_name")\
    .withColumnRenamed("number","driver_number").withColumnRenamed("nationality","driver_nationality")



In [0]:
race_circuits_df=circuits_df.join(races_df,races_df["circuit_id"]==circuits_df["circuit_id"],"inner").select(races_df.race_id,races_df.race_name,races_df.race_year,races_df.race_date,circuits_df.circuit_name,circuits_df.circuit_location)
display(race_circuits_df)

In [0]:
#join with race_circuits_df and Results df
races_results_df=results_df.join(race_circuits_df,race_circuits_df["race_id"]==results_df["result_race_id"],"inner")\
                            .join(drivers_df,drivers_df.driver_id==results_df.result_driver_id)\
                            .join(constructors_df,results_df['result_constructor_id']==constructors_df['constructor_id'])

final_df=races_results_df.select(races_results_df.race_year,races_results_df.race_name,races_results_df.race_date    ,races_results_df.circuit_location,races_results_df.circuit_name,races_results_df.driver_id,races_results_df.driver_name   ,races_results_df.driver_nationality,races_results_df.constructor_name,races_results_df.grid   ,races_results_df.fastest_lap,races_results_df.race_time,races_results_df.points,races_results_df.position)

display(final_df)

In [0]:
final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/formula1_results")

In [0]:
read_formula_df=spark.read.parquet(f"{presentation_folder_path}/formula1_results")
display(read_formula_df.filter("race_year=='2020'").filter("race_name=='Abu Dhabi Grand Prix'").orderBy(read_formula_df["points"].desc()))
